#### Notebooks
|  | Name | Duration* |
| ------ | ------ | ------ |
| 1. | [Introduction](https://colab.research.google.com/github/bertcoerver/pywapor_notebooks/blob/main/pywapor_101.ipynb) | 10 + 30 |
| 2. | [Levels](https://colab.research.google.com/github/bertcoerver/pywapor_notebooks/blob/main/levels.ipynb) | 10 + 120 |
| 3. | **Composites** | 10 + 10 |
| 4. | [pyWaPOR vs. WaPOR](https://colab.research.google.com/github/bertcoerver/pywapor_notebooks/blob/main/wapor_vs_pywapor.ipynb) | 10 + 30 |
| 5. | Soil Saturation | > Coming Soon < |

\* Estimation of the time required in minutes, as in "active" + "download time". 

\* Also note that running the different notebooks on the same instance (or your own computer), will save you a lot of "download time" because the downloaded data can be reused between the notebooks.

# Composites

In this notebook we'll take a closer look at how pre_et_look works internally. To do so, we'll be using a couple of functions that you'd normally not need to touch, because they are called automatically when you pre_et_look.

First, we'll install pywapor in case it's not installed yet.

In [ ]:
!pip install pywapor==2.3.2 --quiet

Then we can define some of the basic parameters, that were explained in the introduction notebook. In this case we'll look at the Fayoum region again, for a period of two months.

In [ ]:
import pywapor

project_folder = r"/Users/hmcoerver/On My Mac/pyWAPOR/example_data"
latlim = [28.9, 29.7]
lonlim = [30.2, 31.2]
startdate = "2021-06-01"
enddate = "2021-08-01"

One of the first things happening when you run pre_et_look, is the creation of a more precise definition of the composites (from here on also referred to as epochs) for which pre_et_look will prepare the data to be used by et_look.

The default value for the composite_length in pywapor.pre_et_look.main is "DEKAD", which creates three composites per month, just like much of the WaPOR data. We'll go with this default value for now. The variable epochs_info consists of three lists, with information on the number, startdate and enddate of each epoch.

In [ ]:
composite_length = "DEKAD"

epochs_info = pywapor.pre_et_look.create_dates(startdate, enddate, composite_length)

for number, start, end in zip(*epochs_info):
    print(f"Composite {number + 1} starts at {start} and ends at {end}.\n")

Besides defining the epochs, pre_et_look also downloads data. In this example we'll look solely at NDVI data (from MOD13 and MYD13). So let's download some data and store it in our RAW folder. In case you already went trough some other notebooks and you're using the same project folder, some of the data should already be there and this step should go pretty quickly.

In [ ]:
import os

raw_folder = os.path.join(project_folder, "RAW")

dl_args = (raw_folder, latlim, lonlim, startdate, enddate)

raw_ndvi_files = list()
raw_ndvi_files.append(pywapor.collect.MOD13.NDVI(*dl_args))
raw_ndvi_files.append(pywapor.collect.MYD13.NDVI(*dl_args))

Now that we have defined our epochs and downloaded some data, we can create our first composites. To do so, we need to define one more variable, which specifies what kind of composite we want to create. This variable, cmeta, is a dictionary with a couple of keys.

* var_name and var_unit simply define the name and unit of the data we are dealing with.
* spatial_interp defines the interpolation method used to align datasets with different resolutions with eachother. Since MOD13 and MYD13 each have the same pixel-size, spatial_interp is not relevant here. Possible values are "nearest" or "linear".
* Finally, composite_type and temporal_interp will be explained in more detail below.

> Also note that we're passing lean_output as False, by doing so the output of this function will be a bit more extensive than normally, i.e. some temporary internal variables that normally get deleted, are stored and returned now, for the sake of this exercise.

In [ ]:
cmeta = {
    "var_name": "ndvi",
    "var_unit": "-",
    "spatial_interp": "nearest",
    "composite_type": "mean",
    "temporal_interp": False,
}

ds = pywapor.general.compositer.main(cmeta, raw_ndvi_files, epochs_info, lean_output = False)

The compositer.main function return an xarray.Dataset object. We can inspect it my simple doing the following.

In [ ]:
ds

You'll see a couple of things here:

* First of all, the dimensions of this dataset.
* Next the different coordinates of the variables.
* Then a list of data variables and their respective dimensions. For example, the variable band_data is a 3-dimensional datacube. This variable contains all the original pixel values from the geoTIFFs we've downloaded earlier. 
* Finally, there is an attribute section, which stores some metadata on this dataset.

It's quite easy to plot data inside xarray.Dataset object. For example we can create a map of one of the composites we have created (the first epoch in this case).

In [ ]:
qm = ds.composite.isel(epoch = 0).plot()

Next, we can have a closer look at the composites we've created, by defining a dictionary with points of interest (POI). Here we'll just add one point to the dictionary and name it "Agriculture", since this point is situated in an agricultural area and we'll also add a dot on the previously created map to see where this point is situated.

In [ ]:
diagnostics = { # label          # lat      # lon
                "Agriculture":	(29.32301,	30.77599),
              }

fig = qm.get_figure()
fig.clf()
qm = ds.composite.isel(epoch = 0).plot()
fig = qm.get_figure()
for poi_name, (lat, lon) in diagnostics.items():
    fig.gca().scatter(lon, lat, c = "black", label = poi_name)
fig.gca().legend()

Now we can plot a timeseries of the NDVI at this POI.

>In case you define more POIs, you might also want define an output folder (trough the keyword argument out_folder), as this will make sure the different plots get saved as png-files.

In [ ]:
pywapor.post_et_look.plot_composite(ds, diagnostics, out_folder = None)

In the graph we have just created, you should see a couple of things. First of all, there are red-stars and green-dots that show available NDVI measurements at our POI, coming from MOD13 and MYD13 respectively. Then there are blue bars that show the composite pixel values at this location. Since our composites are dekadal, the width of the bars is about 10 days (on the x-axis).

Each composite value is the mean of the available measurements within the respective epoch. For example, most of the bars match with some of the red-stars or green-dots. While the first bar, ranging from June 1st till June 11th is exactly in between two measurements, i.e. it's the mean of the available measurements.

The composite type is defined using the keyword composite_type in the cmeta variable. Possible values are "mean", "min", "max" or a float value to calculate a percentile (the percentile option is  significantly more computationally heavy than the others).

Now let's adjust our composite_length to create 6-day composites and see what happends.

In [ ]:
epochs_info = pywapor.pre_et_look.create_dates(startdate, enddate, 6)
for number, start, end in zip(*epochs_info):
    print(f"Composite {number + 1} starts at {start} and ends at {end}.\n")

In [ ]:
ds = pywapor.general.compositer.main(cmeta, raw_ndvi_files, epochs_info, lean_output = False)
pywapor.post_et_look.plot_composite(ds, diagnostics)

In the new graph, you'll see that the blue bars have become smaller, since our epochs are now shorter. As a result, some composites could not be created, because no NDVI measurements were available during that respective epoch. For example, the fourth composite (ranging between 2021-06-19 00:00:00 and 2021-06-25 00:00:00) is missing.

We solve this by applying a temporal interpolation between the available measurements. This is especially suitable for this data, because the MOD13 and MYD13 datasets itself are 16-daily composites, i.e. their measurements can be considered valid for a 16-day period.

In [ ]:
cmeta = {
    "composite_type": "mean",
    "temporal_interp": "nearest",
    "spatial_interp": "nearest",
    "var_name": "ndvi",
    "var_unit": "-",
}

ds = pywapor.general.compositer.main(cmeta, raw_ndvi_files, epochs_info, lean_output = False)
pywapor.post_et_look.plot_composite(ds, diagnostics)

Possible values for temporal_interp are False, "linear", "nearest", "zero", "slinear", "quadratic" or "cubic". But be aware that some of these can be computationally heavy, as you can see when combining a cubic temporal interpolation with a percentile composite_type.

In [ ]:
cmeta = {
    "composite_type": 0.75,
    "temporal_interp": "cubic",
    "spatial_interp": "nearest",
    "var_name": "ndvi",
    "var_unit": "-",
}

ds = pywapor.general.compositer.main(cmeta, raw_ndvi_files, epochs_info, lean_output = False)
pywapor.post_et_look.plot_composite(ds, diagnostics)